In [7]:
import os, sys
from glob import glob
import numpy as np
import dask
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *
from mitequinox.sigp import *
from mitequinox.plot import *

In [8]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=12, walltime='06:00:00')
w = cluster.scale(12*10)

In [9]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [10]:
client

Client Scheduler: tcp://10.135.39.30:38820 Dashboard: http://10.135.39.30:8787/status,Cluster Workers: 120 Cores: 120 Memory: 999.60 GB


_______
# Useful information

In [10]:
E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(8584))

In [7]:
#grd = load_grd(ftype='nc').reset_coords()
grd = load_grd().reset_coords()
mask = ((grd.hFacW.rename({'i_g': 'i'}) == 1) &
        (grd.hFacS.rename({'j_g': 'j'}) == 1) 
       ).rename('mask').reset_coords(drop=True)
grd_rspec = xr.merge([mask, grd])
#print(grd_rspec)

# !! chunking is coarse for the netcdf grid

___________
# Term 1: time acceleration

### du/dt and dv/dt

In [8]:
dij = 4

dsU = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU')).isel(time=slice(200,8784)) #8584
dsV = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV')).isel(time=slice(200,8784))

ds_raw = (xr.merge([dsU,dsV]).assign_coords(**grd.variables))
ds_raw = ds_raw.assign_coords(time=time_day) 
grid = xgcm.Grid(ds_raw, periodic=['X', 'Y'])
print(ds_raw)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 8584)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(8584,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2012-11-14T23:00:00
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    CS       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    PHrefC   (k) float32 dask.array<chunksize=(90,), meta=np.ndarray>
    PHref

In [ ]:
dt = 24

for face in range(ds_raw['face'].size):
    
    d_SSU_dt_time = ( ds_raw['SSU'].isel(face=face).shift(time=-1) - ds_raw['SSU'].isel(face=face).shift(time=1) )/2/3600
    d_SSV_dt_time = ( ds_raw['SSV'].isel(face=face).shift(time=-1) - ds_raw['SSV'].isel(face=face).shift(time=1) )/2/3600
 
    dudt = grid.interp(d_SSU_dt_time.isel(time=slice(0,None,dt)),'X')
    dvdt = grid.interp(d_SSV_dt_time.isel(time=slice(0,None,dt)),'Y')    

    du_dt =  dudt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.CS.isel(face=face).isel(i=slice(0,None,dij), j=slice(0,None,dij)) - dvdt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.SN.isel(face=face).isel(i=slice(0,None,dij), j=slice(0,None,dij))
    dv_dt =  dudt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.SN.isel(face=face).isel(i=slice(0,None,dij), j=slice(0,None,dij)) + dvdt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.CS.isel(face=face).isel(i=slice(0,None,dij), j=slice(0,None,dij))    
    
    #print(du_dt)
    # store
    du_dt = np.real(du_dt).rename('du_dt')    
    du_dt = du_dt.chunk({'time': 1,'i':1080,'j':1080})
    du_dt.to_dataset().to_zarr(E_dir+'du_dt_f%02d.zarr'% (face), mode='w')

    dv_dt = np.real(dv_dt).rename('dv_dt')    
    dv_dt = dv_dt.chunk({'time': 1,'i':1080,'j':1080})
    dv_dt.to_dataset().to_zarr(E_dir+'dv_dt_f%02d.zarr'% (face), mode='w')        
    #print(dv_dt)
    print('face=%d - allready processed'%(face))    

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed


# A key term zeta

In [61]:
# No need to do these for KE, but it's necessary to do for zeta (i.e. no need to ratote uv first to compute zeta)

dij=4
dt=24

#for face in range(13):
for face in [2,3,4,5,6,7,8,9,10,11,12]:

    zeta_3 = (grid.diff(ds_raw.dyC.isel(face=face) * ds_raw.SSV.isel(face=face,time=slice(0,None,dt)),'X') - grid.diff(ds_raw.dxC.isel(face=face) * ds_raw.SSU.isel(face=face,time=slice(0,None,dt)),'Y'))/ds_raw.rAz.isel(face=face) # relative vorticity  

    zeta = grid.interp(grid.interp(zeta_3,'Y'),'X').isel(i=slice(0,None,dij), j=slice(0,None,dij))   #(zeta) at i,j
    zeta = np.real(zeta).rename('zeta')
    zeta = zeta.chunk({'time': 1,'i':1080,'j':1080})
    zeta.to_dataset().to_zarr(E_dir+'zeta_rotation_f%02d.zarr'% (face), mode='w')

    print('face=%d - allready processed'%(face))

/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10

face=2 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10

face=3 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10

...

face=10 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10

face=11 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10

face=12 - allready processed


___________
# Useful info.

In [6]:
E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])
dsU = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU')).isel(time=slice(200,8784)) #8584
dsV = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV')).isel(time=slice(200,8784))
dsE = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('Eta')).isel(time=slice(1512+200,1512+8784))
dsU

<xarray.Dataset>
Dimensions:  (face: 13, i_g: 4320, j: 4320, time: 8584)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(8584,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 6.422e+06 6.426e+06 6.43e+06 ... 3.732e+07 3.732e+07
Data variables:
    SSU      (time, face, j, i_g) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>

In [16]:
# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(8584))

dsE = dsE.assign_coords(time=time_day) 
dsU = dsU.assign_coords(time=time_day) 
dsV = dsV.assign_coords(time=time_day) 
dsU = dsU.assign_coords(**grd.variables)
grid = xgcm.Grid(dsU, periodic=['X', 'Y'])
print(grid)

<xgcm.Grid>
Z Axis (not periodic):
  * center   k --> left
  * left     k_l --> center
  * outer    k_p1 --> center
  * right    k_u --> center
X Axis (periodic):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic):
  * center   j --> left
  * left     j_g --> center


In [17]:
dt = 24

for face in range(dsE['face'].size):
    
    Eta = dsE['Eta'].isel(face=face).isel(time=slice(0,None,dt))    
    Eta = np.real(Eta).rename('Eta')
    Eta = Eta.chunk({'time': 1,'i':4320,'j':4320})
    print(Eta)
    Eta.to_dataset().to_zarr(E_dir+'Eta_f%02d.zarr'% (face), mode='w')
    
    print('face=%d - allready processed'%(face))

<xarray.DataArray 'Eta' (time: 358, j: 4320, i: 4320)>
dask.array<real, shape=(358, 4320, 4320), dtype=float32, chunksize=(1, 4320, 4320), chunktype=numpy.ndarray>
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    face     int64 0
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2012-11-14T08:00:00
face=0 - allready processed
<xarray.DataArray 'Eta' (time: 358, j: 4320, i: 4320)>
dask.array<real, shape=(358, 4320, 4320), dtype=float32, chunksize=(1, 4320, 4320), chunktype=numpy.ndarray>
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    face     int64 1
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.arra

In [18]:
for face in range(dsU['face'].size):
    
    U_ij = grid.interp(dsU['SSU'].isel(face=face).isel(time=slice(0,None,dt)),'X')
    V_ij = grid.interp(dsV['SSV'].isel(face=face).isel(time=slice(0,None,dt)),'Y')
    
    SSU_rotation = U_ij*dsU.CS.isel(face=face) - V_ij*dsU.SN.isel(face=face)
    SSV_rotation = U_ij*dsU.SN.isel(face=face) + V_ij*dsU.CS.isel(face=face)
    
    SSU_rotation = np.real(SSU_rotation).rename('SSU_rotation')
    SSU_rotation = SSU_rotation.chunk({'time': 1,'i':4320,'j':4320})
    print(SSU_rotation)
    SSU_rotation.to_dataset().to_zarr(E_dir+'SSU_rotation_f%02d.zarr'% (face), mode='w')

    SSV_rotation = np.real(SSV_rotation).rename('SSV_rotation')
    SSV_rotation = SSV_rotation.chunk({'time': 1,'i':4320,'j':4320})
    print(SSV_rotation)
    SSV_rotation.to_dataset().to_zarr(E_dir+'SSV_rotation_f%02d.zarr'% (face), mode='w')   
    
    print('face=%d - allready processed'%(face))

<xarray.DataArray 'SSU_rotation' (time: 358, j: 4320, i: 4320)>
dask.array<rechunk-merge, shape=(358, 4320, 4320), dtype=float32, chunksize=(1, 4320, 4320), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2012-11-14T08:00:00
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    face     int64 0
    CS       (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
<xarray.DataArray 'SSV_rotation' (time: 358, j: 4320, i: 4320)>
dask.array<rechunk-merge, shape=(358, 4320, 4320), dtype=float32, chunksize=(1, 4320,

## Read rotated uv

In [7]:
E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

face_all = 13

D = [xr.open_zarr(E_dir+'SSU_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSU = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'SSV_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSV = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'Eta_f%02d.zarr'%(face)) for face in range(face_all)]
ds_Eta = xr.concat(D, dim='face')

E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])

%time ds = xr.merge([ds_SSU, ds_SSV, ds_Eta]).assign_coords(**grd.variables)
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

In [22]:
grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])
grid = xgcm.Grid(grd, periodic=['X', 'Y'])
grid

<xgcm.Grid>
X Axis (periodic):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic):
  * center   j --> left
  * left     j_g --> center
Z Axis (not periodic):
  * center   k --> left
  * left     k_l --> center
  * outer    k_p1 --> center
  * right    k_u --> center

In [9]:
# coriolis term
lat = grd['YC']
omega = 7.3/100000
f_ij = 2*omega*np.sin(np.deg2rad(lat))
f_ij

<xarray.DataArray 'YC' (face: 13, j: 4320, i: 4320)>
dask.array<mul, shape=(13, 4320, 4320), dtype=float32, chunksize=(1, 4320, 4320), chunktype=numpy.ndarray>
Coordinates:
    CS       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    SN       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XC       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319

___________
## Term 2: Coriolis terms

### f*v

In [10]:
dij=4

for face in range(ds['face'].size):

    fv = ds['SSV_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))*f_ij.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) 
    fv = np.real(fv).rename('fv')
    fv = fv.chunk({'time': 1,'i':1080,'j':1080})
    #print(fv)
    fv.to_dataset().to_zarr(E_dir+'fv_f%02d.zarr'% (face), mode='w')
    
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


### f*u

In [12]:
for face in range(ds['face'].size):
#for face in [1]:

    fu = ds['SSU_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))*f_ij.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    fu = np.real(fu).rename('fu')
    fu = fu.chunk({'time': 1,'i':1080,'j':1080})
    fu.to_dataset().to_zarr(E_dir+'fu_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


face=0 - allready processed
face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


face=2 - allready processed
face=3 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)


...

face=8 - allready processed
face=9 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


face=10 - allready processed
face=11 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


face=12 - allready processed


___________
## Term 2.2: zeta*uv

In [18]:
D = [xr.open_zarr(E_dir+'zeta_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_zeta = xr.concat(D, dim='face')
face_13 = range(0,13)
ds_zeta = ds_zeta.assign_coords(face=face_13) 
ds_zeta

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080, time: 20)
Coordinates:
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-11-24T03:00:00
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    zeta     (face, j, i, time) float32 dask.array<chunksize=(1, 1080, 1080, 1), meta=np.ndarray>

In [19]:
dij=4

for face in range(ds['face'].size):

    zeta_v = ds_zeta['zeta'].isel(face=face)*ds['SSV_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    zeta_v = np.real(zeta_v).rename('zeta_v')
    zeta_v = zeta_v.chunk({'time': 1,'i':1080,'j':1080})
    #print(zeta_v)
    %time zeta_v.to_dataset().to_zarr(E_dir+'zeta_v_f%02d.zarr'% (face), mode='w')
    
    print('face=%d - allready processed'%(face))

CPU times: user 414 ms, sys: 77.5 ms, total: 492 ms
Wall time: 816 ms
face=0 - allready processed
CPU times: user 435 ms, sys: 66.2 ms, total: 501 ms
Wall time: 774 ms
face=1 - allready processed
CPU times: user 425 ms, sys: 60.2 ms, total: 485 ms
Wall time: 825 ms
face=2 - allready processed
CPU times: user 425 ms, sys: 56.9 ms, total: 482 ms
Wall time: 763 ms
face=3 - allready processed
CPU times: user 432 ms, sys: 59.7 ms, total: 492 ms
Wall time: 827 ms
face=4 - allready processed
CPU times: user 421 ms, sys: 51.9 ms, total: 473 ms
Wall time: 710 ms
face=5 - allready processed
CPU times: user 422 ms, sys: 56.4 ms, total: 478 ms
Wall time: 755 ms
face=6 - allready processed
CPU times: user 416 ms, sys: 50.9 ms, total: 467 ms
Wall time: 763 ms
face=7 - allready processed
CPU times: user 624 ms, sys: 77 ms, total: 701 ms
Wall time: 1.02 s
face=8 - allready processed
CPU times: user 430 ms, sys: 61.5 ms, total: 491 ms
Wall time: 725 ms
face=9 - allready processed
CPU times: user 422 ms

In [46]:
dij=4

for face in range(ds['face'].size):

    zeta_u = ds_zeta['zeta'].isel(face=face)*ds['SSU_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    zeta_u = np.real(zeta_u).rename('zeta_u')
    zeta_u = zeta_u.chunk({'time': 1,'i':1080,'j':1080})
    #print(zeta_u)
    zeta_u.to_dataset().to_zarr(E_dir+'zeta_u_f%02d.zarr'% (face), mode='w')
    
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


________
# Term 3: Gradient of Bernoulli function

In [28]:
E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

face_all = 13

D = [xr.open_zarr(E_dir+'SSU_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSU = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'SSV_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSV = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'Eta_f%02d.zarr'%(face)) for face in range(face_all)]
ds_Eta = xr.concat(D, dim='face')

E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/daily/'

grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])

%time ds = xr.merge([ds_SSU, ds_SSV, ds_Eta]).assign_coords(**grd.variables)
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


CPU times: user 1min 5s, sys: 19.2 s, total: 1min 24s
Wall time: 1min 27s
<xarray.Dataset>
Dimensions:       (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 358)
Coordinates:
  * i             (i) int64 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
  * time          (time) datetime64[ns] 2011-11-23T08:00:00 ... 2012-11-14T08:00:00
  * j             (j) int64 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
    CS            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth         (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    SN            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XC            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    YC            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
  * face          (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    i

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


In [31]:
for face in range(ds['face'].size):
#for face in [1]:

    KE = (ds['SSU_rotation'].isel(face=face)**2 + ds['SSV_rotation'].isel(face=face)**2)/2 # KE
        
    dB_dx_raw = grid.diff(KE,'X')/ds.dxC.isel(face=face)     # d(KE)/dx
    dB_dy_raw = grid.diff(KE,'Y')/ds.dyC.isel(face=face)     # d(KE)/dy

    # i,j
    dB_dx_raw_ij = grid.interp(dB_dx_raw,'X')
    dB_dy_raw_ij = grid.interp(dB_dy_raw,'Y')
    # rotation 
    dB_dx =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) - dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    dB_dy =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) + dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))

    # store
    dB_dx = np.real(dB_dx).rename('dKE_dx')    
    dB_dx = dB_dx.chunk({'time': 1,'i':1080,'j':1080})
    dB_dx.to_dataset().to_zarr(E_dir+'dKE_dx_f%02d.zarr'% (face), mode='w')
    
    dB_dy = np.real(dB_dy).rename('dKE_dy')    
    dB_dy = dB_dy.chunk({'time': 1,'i':1080,'j':1080})
    dB_dy.to_dataset().to_zarr(E_dir+'dKE_dy_f%02d.zarr'% (face), mode='w')        

    print('face=%d - allready processed'%(face))

distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=0 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

...

distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=11 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=12 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)


In [32]:
# about 1.5 hours

for face in range(ds['face'].size):
#for face in [1]:
    
    g = 9.81

    dB_dx_raw = grid.diff(g*ds['Eta'].isel(face=face),'X')/ds.dxC.isel(face=face)     # d(Eta*g + KE)/dx
    dB_dy_raw = grid.diff(g*ds['Eta'].isel(face=face),'Y') /ds.dyC.isel(face=face)     # d(Eta*g + KE)/dy

    # i,j
    dB_dx_raw_ij = grid.interp(dB_dx_raw,'X')
    dB_dy_raw_ij = grid.interp(dB_dy_raw,'Y')
    # rotation
    dB_dx =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) - dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    dB_dy =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) + dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))

    # store
    dB_dx = np.real(dB_dx).rename('deta_dx')    
    dB_dx = dB_dx.chunk({'time': 1,'i':1080,'j':1080})
    dB_dx.to_dataset().to_zarr(E_dir+'deta_dx_f%02d.zarr'% (face), mode='w')

    dB_dy = np.real(dB_dy).rename('deta_dy')    
    dB_dy = dB_dy.chunk({'time': 1,'i':1080,'j':1080})
    dB_dy.to_dataset().to_zarr(E_dir+'deta_dy_f%02d.zarr'% (face), mode='w')        

    print('face=%d - allready processed'%(face))

distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=0 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

...

distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=11 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

face=12 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


# Term 5: residuals

In [82]:
for face in range(13):
        
    ds_du_dt = xr.open_zarr(E_dir+'du_dt_f%02d.zarr'%face)
    ds_fv = xr.open_zarr(E_dir+'fv_f%02d.zarr'%face)
    ds_zv = xr.open_zarr(E_dir+'zeta_v_f%02d.zarr'%face)
    ds_dKE_dx = xr.open_zarr(E_dir+'dKE_dx_f%02d.zarr'%face)
    ds_deta_dx = xr.open_zarr(E_dir+'deta_dx_f%02d.zarr'%face)
    
    residuals_u = ds_du_dt['du_dt'] - ds_fv['fv'] - ds_zv['zeta_v'] + ds_dKE_dx['dKE_dx'] + ds_deta_dx['deta_dx']
    residuals_u = np.real(residuals_u).rename('residuals_u')
    residuals_u = residuals_u.chunk({'time': 1,'i':1080,'j':1080})
    residuals_u.to_dataset().to_zarr(E_dir+'residuals_u_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


face=0 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


face=1 - allready processed
face=2 - allready processed
face=3 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


...

face=6 - allready processed
face=7 - allready processed
face=8 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


face=9 - allready processed
face=10 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


face=11 - allready processed
face=12 - allready processed


In [83]:
for face in range(13):
        
    ds_dv_dt = xr.open_zarr(E_dir+'dv_dt_f%02d.zarr'%face)
    ds_fu = xr.open_zarr(E_dir+'fu_f%02d.zarr'%face)
    ds_zu = xr.open_zarr(E_dir+'zeta_u_f%02d.zarr'%face)
    ds_dKE_dy = xr.open_zarr(E_dir+'dKE_dy_f%02d.zarr'%face)
    ds_deta_dy = xr.open_zarr(E_dir+'deta_dy_f%02d.zarr'%face)
    
    residuals_v = ds_dv_dt['dv_dt'] + ds_fu['fu'] + ds_zu['zeta_u'] + ds_dKE_dy['dKE_dy'] + ds_deta_dy['deta_dy'] 
    residuals_v = np.real(residuals_v).rename('residuals_v')
    residuals_v = residuals_v.chunk({'time': 1,'i':1080,'j':1080})
    residuals_v.to_dataset().to_zarr(E_dir+'residuals_v_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


face=1 - allready processed
face=2 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


face=3 - allready processed
face=4 - allready processed


...

face=5 - allready processed
face=6 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


face=7 - allready processed
face=8 - allready processed
face=9 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


In [137]:
cluster.close()